<a href="https://colab.research.google.com/github/murillo-borges/webscraping-livelo-esfera/blob/main/Webscraping_Whats_Milhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dados padronizados

### Instalando Bibliotecas


In [1]:
pip install pandas

In [2]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.7/264.7 kB 12.7 MB/s eta 0:00:00


### Importando Bibliotecas


In [3]:
import requests
import pandas as pd
import re
import os
import datetime
import time
from bs4 import BeautifulSoup
from zoneinfo import ZoneInfo
import textwrap

### Horarios envios
- 08h
- 11h
- 14h
- 18h

In [4]:
#Horarios de corte para envios
hora_max_grupo_perfumaria_cosmeticos = 9
hora_min_grupo_esportivo = 11
hora_max_grupo_esportivo = 13
hora_min_grupo_moda = 12
hora_max_grupo_moda = 13
hora_min_grupo_cosmeticos = 14
hora_max_grupo_cosmeticos = 15
hora_min_grupo_varejo = 17
hora_max_grupo_varejo = 18
hora_jabar_18 = 18

### Definindo o ID do Grupo

In [5]:
id_grupo_envio = os.environ['ID_GRUPO_PESSOAL_TESTES'] #os.environ['ID_GRUPO_MILHAS']

KeyError: 'ID_GRUPO_PESSOAL_TESTES'

### Definindo os dados da API para envio no Whatsapp

In [ ]:
INSTANCE_ID = os.environ['INSTANCE_ID']
API_KEY = os.environ['API_KEY']

# Endpoint
url = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/text" #url para mensagens de texto
url_media = f"https://api.zapperapi.com/{INSTANCE_ID}/messages/media" #url para mensagens de texto com imagens

headers = {
    "X-Api-Key": API_KEY,
    "Content-Type": "application/json"
}

### Função para Webscrapping

In [ ]:
#Coletando a data do dia
data_hoje = datetime.datetime.now(ZoneInfo("America/Sao_Paulo")).date()
data_formatada = data_hoje.strftime("%d/%m/%Y")

# Lista de nomes que você quer capturar (normalizados)
nomes_alvo = ["livelo", "shopping livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "pass", "LATAM Pass", "Shopping Livelo"]

#Buscando a hora atual
agora_brasilia = datetime.datetime.now(ZoneInfo("America/Sao_Paulo"))

hora_atual = int(agora_brasilia.strftime("%H"))

def coletar_promocoes(url, parceiro, data_formatada, nomes_alvo):
    pagina = requests.get(url)
    dados_pagina = BeautifulSoup(pagina.text, 'html.parser')

    # --- Captura dos nomes ---
    list_nomes_clubes = []
    for sp in dados_pagina.select("div.d-grid span"):
        nome = sp.get_text(strip=True)
        nome_norm = nome.lower()

        if nome_norm not in nomes_alvo:
            continue

        # Emojis
        if nome_norm in("livelo", "Shopping Livelo"):
            nome = f"🟣 {nome}"
        elif nome_norm == "esfera":
            nome = f"🔴 {nome}"
        elif nome_norm == "smiles":
            nome = f"🟠 {nome}"
        elif nome_norm in ("azul", "shopping latam", "latam"):
            nome = f"🔵 {nome}"
        else:
            nome = nome

        list_nomes_clubes.append(nome)

    # --- Captura da pontuação ---
    list_pontuacao_clubes = []
    tabela_pm = dados_pagina.find('th', string=lambda s: s and 'Pontos e Milhas' in s)
    if tabela_pm:
        table = tabela_pm.find_parent('table')
        for tr in table.select('tbody tr'):
            nome_el = tr.select_one('td .d-grid span')
            if not nome_el:
                continue
            nome_norm = nome_el.get_text(strip=True).lower()

            if nome_norm not in nomes_alvo:
                continue

            tds = tr.find_all('td')
            if len(tds) < 2:
                continue

            texto_ganho = tds[-1].get_text(" ", strip=True)
            m = re.search(r'(\d+(?:[.,]\d+)?)\s*pt', texto_ganho, flags=re.I)
            if not m:
                continue

            valor = m.group(1).replace(',', '.')
            val_float = float(valor)
            val_final = int(val_float) if val_float.is_integer() else val_float

            list_pontuacao_clubes.append(val_final)

    # Monta DataFrame
    df = pd.DataFrame({
        "nome_clube": list_nomes_clubes,
        "pontuacao_clube": list_pontuacao_clubes
    })

    # --- Ajuste de pontuação e análise ---
    list_analise = []
    for index, row in df.iterrows():
        nome_norm = row["nome_clube"].lower()

        if ("livelo" not in nome_norm) and ("esfera" not in nome_norm):
            pontuacao_ajustada = row["pontuacao_clube"] / 1.8
        else:
            pontuacao_ajustada = row["pontuacao_clube"]

        #Analisando a promoção
        if pontuacao_ajustada >= 12:
          d_analise = '⭐⭐⭐⭐⭐ Nível 5 (Excelente / Raro) 😏: Promoção rara, aproveite sem medo!'
        elif pontuacao_ajustada >= 8:
          d_analise = '⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, daqui pra cima já vale muito!'
        elif pontuacao_ajustada >= 6:
          d_analise = '⭐⭐⭐ Nível 3 (Bom) 😉: Bom momento para potencializar compras planejadas, mas se puder aguardar, tem coisa melhor!'
        elif pontuacao_ajustada >= 3:
          d_analise = '⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você realmente já iria comprar, mas é melhor aguardar algo melhor ;)'
        else:
          d_analise = '⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor'

        list_analise.append(d_analise)

    df["analise_pontuacao"] = list_analise
    df["parceiro"] = parceiro
    df["data_coleta_promocao"] = data_formatada

    return df

### Mensagem Bom dia

In [ ]:
#Criando o texto de bom dia

dia_semana = agora_brasilia.strftime("%A")  # Nome do dia em inglês

if dia_semana == "Monday":
    txt_dia_semana = "segundou! 😴"
elif dia_semana == "Tuesday":
    txt_dia_semana = "terçou!"
elif dia_semana == "Wednesday":
    txt_dia_semana = "quartou!"
elif dia_semana == "Thursday":
    txt_dia_semana = "quintou! 🚀"
elif dia_semana == "Friday":
    txt_dia_semana = "sextou! 🔥"
elif dia_semana == "Saturday":
    txt_dia_semana = "sabadou! 😎"
elif dia_semana == "Sunday":
    txt_dia_semana = "domingou! ☀️"
else:
    txt_dia_semana = "vocês estão bem?"

texto_bom_dia = f"""
☀️ *Bom dia milheiros, {txt_dia_semana}*

Antes de começar o dia, segue o nosso critério de análise para você aproveitar as melhores promoções no acúmulo de milhas:

Nós avaliamos cada promoção em 5 níveis, com base no retorno de milhas/pontos:

• *Nível 1 (Ruim) ⭐* Menos que 3 pontos → Sugiro aguardar algo melhor.
• *Nível 2 (Mediano) ⭐⭐* 3 pontos ou mais → Use só se você já iria comprar de qualquer forma.
• *Nível 3 (Bom) ⭐⭐⭐* 6 pontos ou mais → Bom para compras planejadas, mas ainda pode melhorar.
• *Nível 4 (Muito Bom) ⭐⭐⭐⭐* 8 pontos ou mais → Ótima promoção, daqui pra cima já vale muito a pena!
• *Nível 5 (Excelente / Raro) ⭐⭐⭐⭐⭐* 12 pontos ou mais → Promoção rara, dessas que não aparecem toda hora. Aproveite sem medo!

Quanto maior o nível, melhor o custo-benefício para acumular milhas/pontos.

Obs: Em promoções diretamente nas cias aéreas, nos aplicamos uma divisão de 1,80 para igualar as milhas ao pontos (considerando
uma transferência bonificada de 80%)!

Se você está gostando das nossas análises, deixe o seu like nessa mensagem ;)

Ótimo dia a todos! Bora pra cima! 🚀

Att @murilloinvestor
"""

print(texto_bom_dia)


☀️ *Bom dia milheiros, sextou! 🔥*

Antes de começar o dia, segue o nosso critério de análise para você aproveitar as melhores promoções no acúmulo de milhas:

Nós avaliamos cada promoção em 5 níveis, com base no retorno de milhas/pontos:

• *Nível 1 (Ruim) ⭐* Menos que 3 pontos → Sugiro aguardar algo melhor.
• *Nível 2 (Mediano) ⭐⭐* 3 pontos ou mais → Use só se você já iria comprar de qualquer forma.
• *Nível 3 (Bom) ⭐⭐⭐* 6 pontos ou mais → Bom para compras planejadas, mas ainda pode melhorar.
• *Nível 4 (Muito Bom) ⭐⭐⭐⭐* 8 pontos ou mais → Ótima promoção, daqui pra cima já vale muito a pena!
• *Nível 5 (Excelente / Raro) ⭐⭐⭐⭐⭐* 12 pontos ou mais → Promoção rara, dessas que não aparecem toda hora. Aproveite sem medo!

Quanto maior o nível, melhor o custo-benefício para acumular milhas/pontos.

Obs: Em promoções diretamente nas cias aéreas, nos aplicamos uma divisão de 1,80 para igualar as milhas ao pontos (considerando
uma transferência bonificada de 80%)!

Se você está gostando das n

## Grupo de Perfumaria (08h am) GRUPO OK

### Loja Natura

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_natura = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-natura/",
        parceiro="Natura",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_natura)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Natura,30/10/2025
1,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Natura,30/10/2025
2,🔵 Azul,4.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Natura,30/10/2025
3,🟣 Livelo,4.0,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,Natura,30/10/2025
4,🔵 Shopping Latam,3.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Natura,30/10/2025


### Loja Boticário

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_boticario = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-boticario/",
        parceiro="Boticário",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_boticario)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,6,⭐⭐⭐ Nível 3 (Bom) 😉: Bom momento para potencia...,Boticario,30/10/2025
1,🔴 Esfera,2,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Boticario,30/10/2025


### Loja Eudora

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
    df_eudora = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-eudora/",
        parceiro="Eudora",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_eudora)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,2,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Eudora,30/10/2025
1,🔴 Esfera,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Eudora,30/10/2025


### Enviando os dados via Whatsapp

In [ ]:
if hora_atual < hora_max_grupo_perfumaria_cosmeticos:
  # Concatenando os dataframes
  df_final = pd.concat([df_natura, df_boticario, df_eudora], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"💄 *Resumo das Promoções da Categoria de Perfumaria e Cosméticos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n""\n" + msg

  # Mensagem de Bom dia
  message_bom_dia = texto_bom_dia
  message_promo = msg_grupo

  # Payload
  payload = {
      "jid": id_grupo_envio,
      "message": message_bom_dia,
  }

  #Enviando a mensagem de bom dia
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem de bom dia enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")


  # Aguardar 2 minutos
  time.sleep(120)

  # Payload
  payload = {
      "jid": id_grupo_envio, #group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")

Mensagem de bom dia enviada com sucesso para o grupo!
Mensagem com as promoções enviada com sucesso para o grupo!


##

## Grupo de Roupas e Calçados Esportivos (11h am) GRUPO OK

### Loja Centauro

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_centauro = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-centauro/",
        parceiro="Centauro",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_centauro)
else:
    print("Fora do horário!")

Fora do horário!


### Loja NetShoes

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_netshoes = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-netshoes/",
        parceiro="Netshoes",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_netshoes)
else:
    print("Fora do horário!")

Fora do horário!


### Loja New Balance

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_new_balance = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-new-balance/",
        parceiro="New Balance",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_new_balance)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Nike

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_nike = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-nike/",
        parceiro="Nike",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_nike)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Adidas

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
    df_adidas = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-adidas/",
        parceiro="Adidas",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_adidas)
else:
    print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [ ]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo:
  # Concatenando os dataframes
  df_final = pd.concat([df_centauro, df_netshoes, df_new_balance, df_nike, df_adidas], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"👟🏓 *Promoções da Categoria de Roupas e Calçados Esportivos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n\n" + msg

  # Mensagem
  message_promo = msg_grupo

  # Payload
  payload = {
      "jid": id_grupo_envio, #group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## Grupo de Moda (12h am) GRUPO OK

### Loja Renner

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:
    df_renner = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-lojas-renner/",
        parceiro="Lojas Renner",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_renner)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Lojas Renner,31/10/2025
1,🟣 Livelo,10.0,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Lojas Renner,31/10/2025
2,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Lojas Renner,31/10/2025
3,🔵 Shopping Latam,3.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Lojas Renner,31/10/2025
4,🔵 Azul,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Lojas Renner,31/10/2025


### Loja Riachuelo

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:
    df_riachuelo = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-riachuelo/",
        parceiro="Riachuelo",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_riachuelo)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,3,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,Riachuelo,31/10/2025
1,🟣 Livelo,11,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Riachuelo,31/10/2025
2,🔵 Azul,3,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Riachuelo,31/10/2025


### Loja CEA

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:
    df_cea = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-cea/",
        parceiro="CEA",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_cea)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,3,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,CEA,31/10/2025
1,🔴 Esfera,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,CEA,31/10/2025


### Loja Hering

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:
    df_hering = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-hering/",
        parceiro="CEA",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_hering)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,3,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,CEA,31/10/2025


### Loja Dafiti

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:
    df_dafiti = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-dafiti/",
        parceiro="Dafiti",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_dafiti)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Dafiti,31/10/2025
1,🟣 Livelo,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Dafiti,31/10/2025


### Enviando os dados via Whatsapp

In [ ]:
if hora_atual >= hora_min_grupo_moda and hora_atual < hora_max_grupo_moda:

  # Concatenando os dataframes
  df_final = pd.concat([df_renner, df_riachuelo, df_cea, df_hering, df_dafiti], ignore_index=True)

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"👗👔 *Promoções da Categoria de Moda em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n\n" + msg

  # Mensagem
  message_promo = msg_grupo

  # Payload
  payload = {
      "jid": id_grupo_envio,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

  display(df_final)

else:
  print("Fora do horário!")


Mensagem com as promoções enviada com sucesso para o grupo!


,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Renner,31/10/2025
1,🟣 Livelo,10.0,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Renner,31/10/2025
2,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Renner,31/10/2025
3,🔵 Shopping Latam,3.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Renner,31/10/2025
4,🔵 Azul,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Renner,31/10/2025
5,🔴 Esfera,3.0,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,Riachuelo,31/10/2025
6,🟣 Livelo,11.0,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Riachuelo,31/10/2025
7,🔵 Azul,3.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Riachuelo,31/10/2025
8,🟣 Livelo,3.0,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,CEA,31/10/2025
9,🔴 Esfera,1.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,CEA,31/10/2025


## Grupo de Beleza e Cosméticos (14h am) GRUPO OK

### Loja Beleza na Web

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_beleza_web = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-beleza-na-web/",
        parceiro="Beleza na Web",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_beleza_web)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,10,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Beleza na Web,31/10/2025
1,🔴 Esfera,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Beleza na Web,31/10/2025


### Loja Época Cosméticos

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_epoca_cosmeticos = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-epoca-cosmeticos/",
        parceiro="Época Cosméticos",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_epoca_cosmeticos)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Época Cosméticos,31/10/2025
1,🔴 Esfera,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Época Cosméticos,31/10/2025


### Loja Sephora

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_sephora = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-sephora/",
        parceiro="Sephora",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_sephora)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,1,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Sephora,31/10/2025


### Loja Oceane

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
    df_oceane = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-oceane/",
        parceiro="Oceane",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_oceane)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,2,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Oceane,31/10/2025


### Enviando os dados via Whatsapp

In [ ]:
if hora_atual >= hora_min_grupo_cosmeticos and hora_atual < hora_max_grupo_cosmeticos:
  # Concatenando os dataframes
  df_final = pd.concat([df_beleza_web, df_epoca_cosmeticos, df_sephora, df_oceane], ignore_index=True)

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"💄 *Resumo das Promoções da Categoria de Beleza e Cosméticos em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n""\n" + msg

  # Mensagem
  message_promo = msg_grupo

  # Payload
  payload = {
      "jid": id_grupo_envio,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

  display(df_final)

else:
  print("Fora do horário!")


Mensagem com as promoções enviada com sucesso para o grupo!


,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🟣 Livelo,10.0,"⭐⭐⭐⭐ Nível 4 (Muito Bom) 😎: Ótima Promoção, da...",Beleza na Web,31/10/2025
1,🔴 Esfera,1.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Beleza na Web,31/10/2025
2,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Época Cosméticos,31/10/2025
3,🔴 Esfera,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Época Cosméticos,31/10/2025
4,🟣 Livelo,1.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Sephora,31/10/2025
5,🟣 Livelo,2.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Oceane,31/10/2025


## Grupo de Varejo (17h am) GRUPO OK

### Loja Amazon (Latam pass nao esta aparecendo)

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_amazon = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-amazon/",
        parceiro="Amazon",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_amazon)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Ponto

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_ponto = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-ponto/",
        parceiro="Ponto",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_ponto)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Casas Bahia

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_casas_bahia = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-casas-bahia/",
        parceiro="Casas Bahia",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_casas_bahia)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Extra

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_extra = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-extra/",
        parceiro="Extra",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_extra)
else:
    print("Fora do horário!")

Fora do horário!


### Loja Magalu

In [ ]:
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_magalu = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-magazine-luiza/",
        parceiro="Magalu",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_magalu)
else:
    print("Fora do horário!")

,nome_clube,pontuacao_clube,analise_pontuacao,parceiro,data_coleta_promocao
0,🔴 Esfera,1.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Magalu,31/10/2025
1,🟠 Smiles,4.5,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Magalu,31/10/2025
2,🟣 Livelo,4.0,⭐⭐ Nível 2 (Mediano) 🧐: Dá pra usar se você re...,Magalu,31/10/2025
3,🔵 Shopping Latam,3.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Magalu,31/10/2025
4,🔵 Azul,1.0,⭐ Nível 1 (Ruim) 😡: Sugiro aguardar algo melhor,Magalu,31/10/2025


### Loja Mercado Livre

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_mercado_livre = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-mercado-livre/",
        parceiro="Mercado Livre",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_mercado_livre)
else:
    print("Fora do horário!")

Fora do horário!


### Loja FastShop

In [ ]:
#nomes_alvo = ["livelo", "esfera", "smiles", "azul", "shopping latam", "latam", "Shopping Livelo"]
hora_atual = agora_brasilia.hour

if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
    df_fast_shop = coletar_promocoes(
        url="https://www.comparemania.com.br/cashback-fast-shop/",
        parceiro="Fast Shop",
        data_formatada=data_formatada,
        nomes_alvo=nomes_alvo
    )
    display(df_fast_shop)
else:
    print("Fora do horário!")

Fora do horário!


### Enviando os dados via Whatsapp

In [ ]:
if hora_atual >= hora_min_grupo_varejo and hora_atual < hora_max_grupo_varejo:
  # Concatenando os dataframes
  df_final = pd.concat([df_amazon, df_mercado_livre, df_magalu, df_ponto, df_casas_bahia, df_extra, df_fast_shop], ignore_index=True)
  df_final

  # Criando a mensagem do grupo de Beleza e Cosméticos
  msg = ""
  msg_titulo = f"📱💻 *Promoções do Grupo de Varejo em {data_formatada}: 👇*"

  for index, row in df_final.iterrows():
    msg += textwrap.dedent(f"""
  *Parceiro:* {row["parceiro"]}
  *Programa de Fidelidade:* {row["nome_clube"]}
  *Pontuação:* {row["pontuacao_clube"]}
  *Análise da Promoção:* {row["analise_pontuacao"]}

  {"-"*30}""").strip()

  msg_grupo = msg_titulo + "\n""\n" + msg

  # Mensagem
  message_promo = msg_grupo

  # Payload
  payload = {
      "jid": id_grupo_envio, #group_jid,
      "message": message_promo,
  }

  #Enviando as promoções
  response = requests.post(url, json=payload, headers=headers)

  # Resposta
  if response.status_code == 200:
      print("Mensagem com as promoções enviada com sucesso para o grupo!")
  else:
      print(f"Erro ao enviar mensagem: {response.status_code} - {response.text}")

else:
  print("Fora do horário!")


Fora do horário!


## Jabares (Mensagens com Imagens)

### Jabar banco inter (11h am) GRUPO OK

In [ ]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo and dia_semana in ("Wednesday", "Saturday"):

  # Aguardar 2 minutos
  time.sleep(120)

  if dia_semana == "Wednesday":
    message_promo = textwrap.dedent("""
    *Quer começar no mundo dos cartões com o pé direito?*

    O Inter Prime é uma ótima escolha: pontua bem, os pontos não expiram e você ainda
    ganha acesso às Salas VIP de forma gratuita, perfeito para dar um upgrade nas suas viagens.

    Lembrando que a forma mais fácil de conseguir esse cartão é *assinando o plano Duo Gourmet anual* 👇

    💰 *Quero economizar R$30 no plano anual do Duo Gourmet?*

    Utilize o código B1EE1054 para ganhar R$ 30 off na assinatura do plano anual pelo app do Inter!
    👉 Acesse: https://intergo.app/1f148152

    🟠 *Abra a sua conta no Banco Inter com o nosso link e ganhe 200 pontos*

    Utilize o código 0F01D993 e abra a sua conta gratuita no Inter. Ao fazer isso, você já receberá 200 pontos Loop de presente!
    👉 https://intergo.app/242d85c4

    -----------------------------

    💳 Não conhece esse cartão? Assista o vídeo abaixo para conhecer os principais benefícios do cartão Inter Prime 👇
    https://www.youtube.com/playlist?list=PLywCMnijM298ruCE-Cey18WvsoV2UqqQf""").strip()

  else:
    message_promo = textwrap.dedent("""
    *Pensando em começar no mundo dos cartões? 💳*

    O Inter Prime é uma excelente opção: boa pontuação, pontos que não expiram e acesso às Salas VIP ✈️🔥

    A maneira mais simples de conseguir esse cartão é assinando o plano Duo Gourmet anual 👇

    💰 *Quer economizar R$30 no plano anual do Duo Gourmet?*
    Use o código **B1EE1054** no app do Inter
    👉 https://intergo.app/1f148152

    🟠 *Ainda não tem conta? abra sua conta no Banco Inter e ganhe 200 pontos Loop*
    Use o código **0F01D993**
    👉 https://intergo.app/242d85c4

    -----------------------------

    💳 Conheça os benefícios do cartão Inter Prime:
    https://www.youtube.com/playlist?list=PLywCMnijM298ruCE-Cey18WvsoV2UqqQf""").strip()

  drive_id = "1OpOK2Ee1q2yJO-UDhfP3w3hFh2uM7y0s"
  img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  payload = {
      "jid": id_grupo_envio, #group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,  # ← usa seu texto de análise
      "filename": "analise.jpg"
  }

  response = requests.post(url_media, json=payload, headers=headers)

  print(response.json())

else:
  print("Fora do horário!")

Fora do horário!


### Jabar planilha (11h am) GRUPO OK

In [ ]:
if hora_atual >= hora_min_grupo_esportivo and hora_atual < hora_max_grupo_esportivo and dia_semana in ("Thursday", "Sunday"):

  # Aguardar 2 minutos
  time.sleep(120)

  if dia_semana == "Thursday":
    message_promo = textwrap.dedent("""
    *Quer organizar suas finanças de um jeito simples e inteligente?*

    A nossa Planilha de Controle Financeiro foi feita para quem quer parar de gastar no automático
    e finalmente entender para onde o dinheiro está indo, de forma clara, rápida e sem complicação.

    Por que essa planilha é a solução perfeita para você?
    • Simples de usar e totalmente automatizada
    • Visão completa do seu fluxo de caixa
    • Dashboard visual e intuitivo

    💡 *E o melhor?*
    • Ela custa menos do que uma pizza 🍕
    • Você paga *uma única vez* e fica com a planilha *para sempre*.
    • E você ainda ganha uma aula passo a passo. Mesmo sem saber Excel, você vai conseguir usar sem dificuldade. 🎓✅
    • Ao adquirir, você ainda apoia o nosso trabalho e nos ajuda a continuar trazendo conteúdo gratuito aqui no grupo. 🙏

    👉 Assista ao vídeo e adquira sua planilha aqui:
    https://bit.ly/whatsplancontrolefinanceiro

    -----------------------------

    🧠 *Organização financeira muda tudo.*
    Comece hoje, seu “eu do futuro” vai agradecer.""").strip()
  else:
    message_promo = textwrap.dedent("""
    *Quer organizar suas finanças de um jeito simples?*

    Nossa Planilha de Controle Financeiro te ajuda a ver para onde seu dinheiro está indo e a controlar seus gastos sem complicação.

    • Custa menos que uma pizza 🍕
    • Pagou uma vez, é sua para sempre.
    • E tem aula passo a passo (mesmo se você não souber Excel). 🎓✅

    👉 Conheça e adquira aqui:
    https://bit.ly/whatsplancontrolefinanceiro""").strip()


  # Imagem do Google Drive
  drive_id = "1dHHTOXW8g3bZ4AGZk14Lvnu9mGSpHoVc"
  img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  payload = {
      "jid": id_grupo_envio, #group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,  # ← usa seu texto de análise
      "filename": "analise.jpg"
  }

  response = requests.post(url_media, json=payload, headers=headers)

  print(response.json())

else:
  print("Fora do horário!")

Fora do horário!


### Jabar cartao Ruby Stell (18h Sexta-Feira e Terça-Feira)

In [ ]:
if hora_atual == hora_jabar_18 and dia_semana in ("Friday", "Tuesday"):

  # Aguardar 1 minuto
  time.sleep(60)

  if dia_semana == "Friday":

    # Imagem do Google Drive
    drive_id = "1EV2mu8AfLkIzo9dM23nP58e5lN7bYN18" #Imagem de Sexta
    img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

    message_promo = textwrap.dedent("""
    *Vai viajar para fora do Brasil ou fazer uma compra internacional? Então presta atenção nisso 👇*

    O *Cartão Crypto Ruby* é simplesmente um dos melhores cartões para uso internacional, porque:
    • Não cobra IOF nas compras (3,5% de economia) 🌍
    • Tem spread super baixo 💱
    • *2% de cashback* em todas as compras 💸
    • Spotify grátis 🎧

    Ou seja: você gasta menos, economiza nas taxas e ainda recebe dinheiro de volta.

    -----------------------------

    💳 *Quer ver esse cartão funcionando na prática?*
    Eu testei esse cartão por 15 dias e gravei um vídeo mostrando como usar, benefícios e pontos de atenção 👇
    https://www.youtube.com/watch?v=OA7LlqiobHQ&t=300s

    -----------------------------

    🔵 Gostou do cartão?
    Abra sua conta na Crypto com o nosso link e ganhe até **$60 USD** de bônus 💵
    👉 https://bit.ly/wcryptoruby

    """).strip()

  else:

    # Imagem do Google Drive
    drive_id = "1eLpOE5cJJe2ahFIMbbY1do2RSE-adggr" #Imagem de Terça
    img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

    message_promo = textwrap.dedent("""
    *Se você vai viajar ou costuma comprar fora do Brasil, esse cartão pode te fazer economizar MUITO* ✈️💸

    O *Crypto Ruby* é um dos poucos cartões que realmente vale a pena no uso internacional:

    • Zero IOF nas compras (economia 3,5%) 🌍
    • Spread super baixo (conversão mais justa) 💱
    • *2% de cashback* em tudo que você gastar 💸
    • Spotify grátis 🎧

    👀 *Spoiler:* ele tem um custo muito menor que vários cartões globais famosos do mercado.

    -----------------------------

    🎥 *Quer ver como ele funciona na prática?*
    Eu usei esse cartão e gravei um vídeo mostrando tudo:
    https://www.youtube.com/watch?v=OA7LlqiobHQ&t=300s

    -----------------------------

    🔵 Curtiu a ideia?
    Abra sua conta pela Crypto e ganhe até *$60 USD* de bônus 💵
    👉 https://bit.ly/wcryptoruby
    """).strip()

  payload = {
      "jid": id_grupo_envio, #group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,
      "filename": "analise.jpg"
  }

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  response = requests.post(url_media, json=payload, headers=headers)

  print("Mensagem enviada!")

else:
  print("Fora do horário!")

{'status': 'scheduled', 'jobId': '101'}


### Jabar TopCashback

In [ ]:
if 1==1: #hora_atual == hora_jabar_18 and dia_semana in ("Friday", "Tuesday"):

  # Aguardar 30 segundos
  time.sleep(30)

  if dia_semana == "Monday":

    # Imagem do Google Drive
    drive_id = "1sVCoiFDKu0gbioPQSpZbOVkGbq6NYb8X" #Imagem de Segunda
    img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  else:

    # Imagem do Google Drive
    drive_id = "1idhL58jQID5wemKh0XoHfqROn7L3MzTK" #Imagem de Quinta
    img_url = f"https://drive.google.com/uc?export=view&id={drive_id}"

  message_promo = textwrap.dedent("""
  🇬🇧 *Vai viajar ou reservar serviços no exterior? Então presta atenção nisso 👇*

  Alguns

  Se você ainda não tem conta no *TopCashBack do Reino Unido*, vale muito a pena abrir.
  Usando o nosso link, você ainda pode ganhar um *bônus após a primeira compra* 💷✨

  👉 Abra sua conta aqui:
  https://bit.ly/topcashbackreinounido

  -----------------------------

  🏨 *Hospedagens com Cashback*
  • ALL Accor Live Limitless – até *15% de cashback* 🏨
    https://www.topcashback.co.uk/all-accor-live-limitless/

  • Hotels.com – até *15% de cashback* 🏨
    https://www.topcashback.co.uk/hotelscom/

  • Expedia – até *8,5% de cashback* ✈️
    https://www.topcashback.co.uk/search/merchants/?s=Expedia

  • Agoda – até *4% de cashback* 🌍
    https://www.topcashback.co.uk/search/merchants/?s=Agoda

  • Lastminute.com – até *8,5% de cashback* ⏰
    https://www.topcashback.co.uk/search/merchants/?s=lastminute.com

  -----------------------------

  🚗 *Locadoras de Carros*
  • Avis – até *9,35% de cashback* 🚘
    https://www.topcashback.co.uk/search/merchants/?s=Avis

  • Discover Cars – até *25,5% de cashback* 🔥
    https://www.topcashback.co.uk/discover-car-hire/

  • Europcar – até *11,47% de cashback* 🚗
    https://www.topcashback.co.uk/europcar/

  • Sixt UK – até *8,5% de cashback* 🚙
    https://www.topcashback.co.uk/sixt-uk/

  • Alamo – até *4,25% de cashback* 🚐
    https://www.topcashback.co.uk/alamo/

  -----------------------------

  🚆 *Trem, Ferry e Ônibus*
  • Omio – até *4,25% de cashback* 🚆
    https://www.topcashback.co.uk/omio/

  -----------------------------

  🎟️ *Passeios e Atrações*
  • GetYourGuide – até *11% de cashback* 🗺️
    https://www.topcashback.co.uk/getyourguide/

  • Groupon – até *5,95% de cashback* 💸
    https://www.topcashback.co.uk/groupon/

  • Tripadvisor – até *12% de cashback* ⭐
    https://www.topcashback.co.uk/tripadvisor-tours-and-experiences/

  • Disneyland Paris – até *£42.50 de cashback* 🎢✨
    https://www.topcashback.co.uk/disneyland-resort-paris/

  -----------------------------

  ⚠️ *Importante:* sempre leia os termos do cashback antes de comprar.
  O percentual pode variar diariamente e cada parceiro possui regras específicas para a validação do retorno.
    """).strip()



  payload = {
      "jid": id_grupo_envio, #group_jid,
      "mediaType": "image",
      "mimetype": "image/jpeg",
      "media": img_url,
      "caption": message_promo,
      "filename": "analise.jpg"
  }

  headers = {
      "X-Api-Key": API_KEY,
      "Content-Type": "application/json"
  }

  response = requests.post(url_media, json=payload, headers=headers)

  print("Mensagem enviada!")

else:
  print("Fora do horário!")